# Noncontextual Electronic Structure

How to solve $H_{noncontextual}$ given the following partition:

$$H_{full} = H_\mathrm{noncontextual} + H_\mathrm{context}$$

# First we need a Hamiltonian

In [1]:
import os
import json

cwd = os.getcwd()
notebook_dir =  os.path.dirname(cwd)
symmer_dir = os.path.dirname(notebook_dir)
test_dir = os.path.join(symmer_dir, 'tests')
ham_data_dir = os.path.join(test_dir, 'hamiltonian_data')

if not os.path.isdir(ham_data_dir):
    raise ValueError('cannot find data dir')

In [2]:
# os.listdir(ham_data_dir)

In [3]:
filename = 'B_STO-3G_DOUBLET_JW.json'#'H2O_STO-3G_SINGLET_BK.json'

if filename not in os.listdir(ham_data_dir):
    raise ValueError('unknown file')
    
with open(os.path.join(ham_data_dir, filename), 'r') as infile:
    data_dict = json.load(infile)

In [4]:
from symmer.symplectic import PauliwordOp
H = PauliwordOp.from_dictionary(data_dict['hamiltonian'])
H.n_qubits

10

# Solving the noncontexutal problem!

In [5]:
from symmer.projection import ContextualSubspace

cs_vqe = ContextualSubspace(H, noncontextual_strategy='SingleSweep_magnitude')

Noncontextual Hamiltonians have a very specific form, namely their terms $\mathcal{T}$ may be decomposed as
\begin{equation}
    \mathcal{T} = \mathcal{S} \cup \mathcal{C}_1 \cup \dots \cup \mathcal{C}_M 
\end{equation}
where $\mathcal{S}$ contains the terms that commute globally (i.e. the noncontextual symmetries) and a collection of cliques with respect to commutation.

In [6]:
cs_vqe.noncontextual_operator.decomposed

{'symmetry': -15.808+0.000j IIIIIIIIII +
  3.977+0.000j ZIIIIIIIII +
  0.198+0.000j IIZIIIIIII +
  0.186+0.000j ZIIIIIIIIZ +
  0.186+0.000j ZIIIIIIZII +
  0.186+0.000j ZIIIIZIIII +
  0.180+0.000j ZIIIIIZIII +
  0.180+0.000j ZIIIIIIIZI +
  0.180+0.000j ZIIIZIIIII +
  0.171+0.000j ZIZIIIIIII +
  0.171+0.000j IZIZIIIIII +
  0.147+0.000j IIIIIIZZII +
  0.147+0.000j IIIIZZIIII +
  0.147+0.000j IIIIIIIIZZ +
  0.133+0.000j IIZIIZIIII +
  0.133+0.000j IIZIIIIZII +
  0.133+0.000j IIZIIIIIIZ +
  0.131+0.000j IIIIIIZIIZ +
  0.131+0.000j IIIIIIIZZI +
  0.131+0.000j IIIIZIIZII +
  0.131+0.000j IIIIZIIIIZ +
  0.131+0.000j IIIIIZZIII +
  0.131+0.000j IIIIIZIIZI +
  0.123+0.000j IIIIIIZIZI +
  0.123+0.000j IIIIZIZIII +
  0.123+0.000j IIIIIIIZIZ +
  0.123+0.000j IIIIZIIIZI +
  0.123+0.000j IIIIIZIZII +
  0.123+0.000j IIIIIZIIIZ +
  0.105+0.000j IIZIZIIIII +
  0.105+0.000j IIZIIIZIII +
  0.105+0.000j IIZIIIIIZI +
  0.050+0.000j IIIIZIIIII +
  0.050+0.000j IIIIIIIZII +
  0.050+0.000j IIIIIIZIII +
  0.050

In [7]:
from symmer.symplectic import NoncontextualOp
H_noncon = NoncontextualOp.from_PauliwordOp(cs_vqe.noncontextual_operator)
H_noncon

-15.808+0.000j IIIIIIIIII +
 3.977+0.000j IZIIIIIIII +
 3.977+0.000j ZIIIIIIIII +
 0.723+0.000j ZZIIIIIIII +
 0.198+0.000j IIZIIIIIII +
 0.198+0.000j IIIZIIIIII +
 0.186+0.000j ZIIZIIIIII +
 0.186+0.000j IZZIIIIIII +
 0.186+0.000j IZIIIIZIII +
 0.186+0.000j IZIIIIIIZI +
 0.186+0.000j ZIIIIIIIIZ +
 0.186+0.000j ZIIIIIIZII +
 0.186+0.000j IZIIZIIIII +
 0.186+0.000j ZIIIIZIIII +
 0.180+0.000j IZIIIIIZII +
 0.180+0.000j ZIIIIIZIII +
 0.180+0.000j IZIIIIIIIZ +
 0.180+0.000j IZIIIZIIII +
 0.180+0.000j ZIIIIIIIZI +
 0.180+0.000j ZIIIZIIIII +
 0.171+0.000j ZIZIIIIIII +
 0.171+0.000j IZIZIIIIII +
 0.147+0.000j IIIIIIZZII +
 0.147+0.000j IIIIZZIIII +
 0.147+0.000j IIIIIIIIZZ +
 0.133+0.000j IIZIIZIIII +
 0.133+0.000j IIZIIIIZII +
 0.133+0.000j IIZIIIIIIZ +
 0.133+0.000j IIIZZIIIII +
 0.133+0.000j IIIZIIZIII +
 0.133+0.000j IIIZIIIIZI +
 0.132+0.000j IIZZIIIIII +
 0.131+0.000j IIIIIIZIIZ +
 0.131+0.000j IIIIIIIZZI +
 0.131+0.000j IIIIZIIZII +
 0.131+0.000j IIIIZIIIIZ +
 0.131+0.000j IIIIIZZIII +


# Solving the noncontexutal problem!

All terms in $H_{noncon}$ can be generated under the jordan product: $P_{a}\cdot P_{b} = \frac{1}{2} \{P_{a}, P_{b} \}$. This ensures only one term from the anticommuting set can be used, as joint assignement to anticommuting Pauli operator is not possible!


Identifying a generating set $\mathcal{G}$ for the symmetry terms $\mathcal{S}$ and constructing the clique operator $A(\vec{r}) = \sum_{i=1}^M r_i C_i$ for clique representatives $C_i \in \mathcal{C}_i$ and coefficients $\vec{r} \in \mathbb{R}^M$ satisfying $|\vec{r}|=1$ allows us to rewrite

\begin{equation}
    H_\mathrm{noncon} = \sum_{P \in \overline{\mathcal{G}}} \bigg(h_{P}^\prime + \sum_{i=1}^M h_{P,i} C_i \bigg) P,
\end{equation}

and yields a classical objective function over parameters $\vec{\nu} \in \{\pm 1\}^{|\mathcal{G}|}$ and $\vec{r} \in \mathbb{R}^M$ for the noncontetual energy expectation value:

\begin{equation}\label{classical_objective}
\begin{aligned}
    \eta(\vec{\nu}, \vec{r}) 
    :={} & {\langle H_\mathrm{noncon} \rangle_{(\vec{\nu}, \vec{r})}} \\
    ={} & \sum_{P \in \overline{\mathcal{G}}} \bigg(h_{P}^\prime + \sum_{i=1}^M h_{P,i} \langle{C_i}\rangle_{(\vec{\nu}, \vec{r})} \bigg) \langle{P}\rangle_{(\vec{\nu}, \vec{r})} \\
    ={} & \sum_{P \in \overline{\mathcal{G}}} \bigg(h_{P}^\prime + \sum_{i=1}^M h_{P,i} r_i \bigg) \prod_{G \in \mathcal{G}_{P}} \nu_{f(G)}.
\end{aligned}
\end{equation}

### Different optimization strategies to find the ground state!

In [8]:
from symmer.utils import exact_gs_energy

gs_energy, psi = exact_gs_energy(H_noncon.to_sparse_matrix)

In [9]:
i=8
print(psi.dagger * H_noncon * psi)
print(psi.dagger * H_noncon.symmetry_generators[i].dagger * H_noncon * H_noncon.symmetry_generators[i] * psi)

(-24.148988598855517-1.700029006457271e-16j)
(-24.148988598855517-1.700029006457271e-16j)


In [10]:
import numpy as np
np.sum(np.array([False, True, False]))

1

In [11]:
# %%timeit
H_noncon.solve(strategy='brute_force', ref_state=psi)
H_noncon.energy, H_noncon.symmetry_generators

/home/tweaving/anaconda3/envs/symmer/lib/python3.8/site-packages/symmer/symplectic/independent_op.py:245: UserWarning: The stabilizers ['IIIIIZIIII'] were assigned zero values - bad reference state.
  warnings.warn(f'The stabilizers {S_zero} were assigned zero values - bad reference state.')


(-24.148988598853645,
  1 IIIIIIIIIZ 
  1 IIIIIIIIZI 
  1 IIIIIIIZII 
  1 IIIIIIZIII 
 -1 IIIIIZIIII 
  1 IIIIZIIIII 
 -1 IIZIIIIIII 
  1 IZIZIIIIII 
 -1 ZIIIIIIIII)

In [12]:
# %%timeit
H_noncon.solve(strategy='binary_relaxation', ref_state=psi)
H_noncon.energy

/home/tweaving/anaconda3/envs/symmer/lib/python3.8/site-packages/symmer/symplectic/independent_op.py:245: UserWarning: The stabilizers ['IIIIIZIIII'] were assigned zero values - bad reference state.
  warnings.warn(f'The stabilizers {S_zero} were assigned zero values - bad reference state.')


-23.94847036640804

In [13]:
# PUSO = Polynomial unconstrained spin Optimization
# QUSO: Quadratic Unconstrained Spin Optimization

In [14]:
H_noncon.symmetry_generators

 1 IIIIIIIIIZ 
 1 IIIIIIIIZI 
 1 IIIIIIIZII 
 1 IIIIIIZIII 
 1 IIIIIZIIII 
 1 IIIIZIIIII 
-1 IIZIIIIIII 
 1 IZIZIIIIII 
-1 ZIIIIIIIII

In [15]:
# %%timeit
H_noncon.solve(strategy='brute_force_PUSO', ref_state=psi)
H_noncon.energy#, H_noncon.symmetry_generators

/home/tweaving/anaconda3/envs/symmer/lib/python3.8/site-packages/symmer/symplectic/independent_op.py:245: UserWarning: The stabilizers ['IIIIIZIIII'] were assigned zero values - bad reference state.
  warnings.warn(f'The stabilizers {S_zero} were assigned zero values - bad reference state.')


-24.148988598853556

In [16]:
H_noncon.symmetry_generators.coeff_vec

array([ 1,  1,  1,  1, -1,  1, -1,  1, -1])

In [17]:
# %%timeit
H_noncon.solve(strategy='brute_force_QUSO', ref_state=psi)
H_noncon.energy

/home/tweaving/anaconda3/envs/symmer/lib/python3.8/site-packages/symmer/symplectic/independent_op.py:245: UserWarning: The stabilizers ['IIIIIZIIII'] were assigned zero values - bad reference state.
  warnings.warn(f'The stabilizers {S_zero} were assigned zero values - bad reference state.')


-24.148988598853556

In [18]:
# %%timeit
H_noncon.solve(strategy='annealing_PUSO', num_anneals=100, ref_state=psi)
H_noncon.energy

/home/tweaving/anaconda3/envs/symmer/lib/python3.8/site-packages/symmer/symplectic/independent_op.py:245: UserWarning: The stabilizers ['IIIIIZIIII'] were assigned zero values - bad reference state.
  warnings.warn(f'The stabilizers {S_zero} were assigned zero values - bad reference state.')
/home/tweaving/anaconda3/envs/symmer/lib/python3.8/site-packages/symmer/symplectic/noncontextual_op.py:511: QUBOVertWarning: The input problem has degree <= 2; consider using the ``qubovert.sim.anneal_qubo`` or ``qubovert.sim.anneal_quso`` functions, which are significantly faster than this function because they take advantage of the low degree.
  puso_res = qv.sim.anneal_puso(spin_problem, num_anneals=self.num_anneals)
/home/tweaving/anaconda3/envs/symmer/lib/python3.8/site-packages/symmer/symplectic/noncontextual_op.py:511: QUBOVertWarning: The input problem has degree <= 2; consider using the ``qubovert.sim.anneal_qubo`` or ``qubovert.sim.anneal_quso`` functions, which are significantly faster t

-24.148988598853556

In [19]:
# %%timeit
ref = psi.cleanup(zero_threshold=1e-4).sort()[0].normalize
print(ref)

H_noncon.solve(strategy='annealing_QUSO', num_anneals=10, ref_state=ref)
H_noncon.energy - gs_energy

 0.168+0.986j |1111010000>


1.822542117224657e-12

In [20]:
H_noncon.symmetry_generators.update_sector(ref)
H_noncon.symmetry_generators

 1 IIIIIIIIIZ 
 1 IIIIIIIIZI 
 1 IIIIIIIZII 
 1 IIIIIIZIII 
-1 IIIIIZIIII 
 1 IIIIZIIIII 
-1 IIZIIIIIII 
 1 IZIZIIIIII 
-1 ZIIIIIIIII

In [21]:
# # brute_force_PUSO
# energy, nu, r = H_noncon._energy_via_brute_force_xUSO(x='P')
# energy

In [22]:
# # brute_force_QUSO
# energy, nu, r = H_noncon._energy_via_brute_force_xUSO(x='Q')
# energy

In [23]:
# # annealing_QUSO
# energy, nu, r = H_noncon._energy_via_annealing_xUSO(x='Q', num_anneals=1000)
# energy

In [24]:
# # annealing_PUSO
# energy, nu, r = H_noncon._energy_via_annealing_xUSO(x='P', num_anneals=100)
# energy